In [1]:
#v2 增加神经网络输出
#v3 修改reward为直接变化量
#v4 删除e,S特征 
#v5 固定贪婪算法阈值0.2

import math
import random
import numpy as np
import pandas as pd

from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
import time

import collections
import copy

#env = gym.make('CartPole-v0').unwrapped

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# if gpu is to be used
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device


device(type='cuda', index=0)

In [2]:

data_path = '/home/ubuntu/data/dataset/ml-100k/'
UIT = pd.read_csv(data_path + 'ml_header.trace')

trainUIT = UIT[UIT['day']<18]
validUIT = UIT[UIT['day']<24]
contentNum = len(UIT.i.drop_duplicates())
userNum = len(UIT.u.drop_duplicates())
contentNum,userNum,trainUIT.shape,validUIT.shape
import os
pathName = '/home/ubuntu/data/PEC/model_dict/ml100k/'
if not os.path.exists(pathName):
    os.makedirs(pathName)
MODELPATH = pathName + 'dnn_o0l0_v5_'

rewardPara = {"alpha":1,"betao":0,"betal":0}
latency = [0.2,1,0.8]
Bu = 15

BATCH_SIZE = 128
#GAMMA = 0.99

num_episodes = 20
#EPS_START = 0.99
#EPS_END = 0.01
#EPS_DECAY = trainUIT.shape[0]*3
#agentStep = 0
eps_threshold = 0.2


In [3]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward','content'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

class DQN(nn.Module):

    def __init__(self,inputs,outputs):
        super(DQN, self).__init__()
        self.conn1 = nn.Linear(inputs,32)
        self.conn2 = nn.Linear(32,128)
        self.conn3 = nn.Linear(128,16)
        self.conn4 = nn.Linear(16, outputs)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = x.float().to(device)
        x = F.relu(self.conn1(x))
        x = F.relu(self.conn2(x))
        x = F.relu(self.conn3(x))
        return self.conn4(x)

In [4]:
class ENV(object):
    def __init__(self,userNum,contentNum,latency,Bu):
        self.userNum = userNum
        self.contentNum =contentNum

        self.r = np.zeros(shape=(userNum,contentNum),dtype=int)
        self.p = np.full(shape=contentNum,fill_value = 1/userNum)
        self.e = np.zeros(shape=contentNum)
        self.S = np.ones(shape=contentNum,dtype=int)
        self.l = np.array(latency)
        

        self.B = np.full(shape=userNum,fill_value=Bu,dtype=int)

        self.pipe = collections.OrderedDict()


    #有序字典实现LRU
    def updateEgdeCache(self,action,t):
        for i in np.argwhere(action==1).squeeze(-1):
            if i in self.pipe.keys():
                self.pipe.pop(i)
            elif len(self.pipe) >= 500:
                self.e[self.pipe.popitem(last=False)[0]] = 0
            self.pipe[i] = t
            self.e[i] = 1

    
    def updateEnv(self,u,action,t):
        
        p_tmp = ((self.r[u] | action)-self.r[u])*(1/self.userNum) + self.p
        self.p = np.where(p_tmp<1-1/self.userNum,p_tmp,1-1/self.userNum)
 
        self.r[u] = self.r[u] | action

        self.updateEgdeCache(action,t)

    def getStatus(self):
        return (torch.from_numpy(self.r),
                torch.from_numpy(self.p), 
                torch.from_numpy(self.e),
                torch.from_numpy(self.S),
                torch.from_numpy(self.l))

    #def reset(self):
    #    self.r = np.zeros(shape=(self.userNum,self.contentNum),dtype=int)
    #    self.p = np.full(shape=self.contentNum,fill_value = 1/self.userNum)
    #    self.e = np.zeros(shape=self.contentNum)
    #    self.S = np.ones(shape=self.contentNum,dtype=int)
    #    self.l_edge = 0.1
    #    self.l_cp = 1
    #    self.B = np.full(shape=self.userNum,fill_value=15,dtype=int)
    #    self.pipe = collections.OrderedDict()


In [5]:
class UE(object):
    def __init__(self,u,env,rewardPara):
        self.u = u

        self.W = []
        self.v = torch.zeros(size=(env.contentNum,),dtype=int)

        self.Bu = int(env.B[self.u])
        self.contentNum = env.contentNum
        self.userNum = env.userNum

        self.r , self.p , self.e, self.S, self.l = env.getStatus()

        self.action = torch.zeros(size=(env.contentNum,),dtype=int)
        self.lastAction = self.action

        self.reward = 0
        self.ALPHAh = rewardPara['alpha']
        self.BETAo =  rewardPara['betao']
        self.BETAl =  rewardPara['betal']             
        self.statusFeature = self.statusEmbedding()

    def updateViewContent(self,i):
        self.W.append(i)
        self.v[i] = 1


    def statusEmbedding(self):
        #statusFeature = torch.zeros(size=(5,env.contentNum)).to(device)
        statusFeature = torch.zeros(size=(3,env.contentNum)).to(device)
        
        statusFeature[0] = self.v
        statusFeature[1] = self.r[self.u]
        statusFeature[2] = self.p
        #statusFeature[3] = self.e
        #statusFeature[4] = self.S

        #statusFeature[5] = status['r']
        return statusFeature.T
    
    def getReward(self,lastru,lastp,ru,p,i,action,S,l,e,v):
        
        #self.Rh =   self.ALPHAh * (torch.log(v * p + (1-v) * (1-p)).sum() / torch.log(ru * p + (1-ru) * (1-p)).sum() )

        #self.Rh = self.ALPHAh * ( 1 / ( 1 + torch.exp( torch.log( ( lastru * lastp + ( 1 - lastru ) * ( 1 - lastp ) ) / ( ru * p + ( 1 - ru ) * ( 1 - p ) ) ).sum() ) ) )

        self.Rh = self.ALPHAh * ( torch.log( ( lastru * lastp + ( 1 - lastru ) * ( 1 - lastp ) ) / ( ru * p + ( 1 - ru ) * ( 1 - p ) ) ).sum() )

        #self.Rh =   self.ALPHAh * torch.log(ru * p + (1-ru) * (1-p))

        self.Ro =   self.BETAo * action[i] * S[i] * ( e[i] * l[0] + ( 1 - e[i] ) * l[1] )
        
        # self.Ro =   self.BETAo * action[i] * S[i] * ( 1 + e[i] * l[0] + ( 1 - e[i] ) * l[1] )

        self.Rl =   self.BETAl * ( 1 - action[i] ) * S[i] * e[i] * l[2]

        #self.Rh[i] = self.Rh[i] + self.Ro + self.Rl

        #return  self.Rh.sum()
        return  self.Rh + self.Ro + self.Rl

    def selectAction(self,env,uit,QNetwork,train,memory):
        
        self.updateViewContent(uit[1])
        
        self.lastStatusFeature = self.statusFeature
        self.lastAction = self.action
        self.lastp = self.p
        self.lastr = self.r

        self.r , self.p , self.e, self.S, self.l = env.getStatus()
        self.statusFeature = self.statusEmbedding()

        if self.W[-1] not in np.argwhere(self.lastAction.numpy()): #local cache miss
            sample = random.random()
            #eps_threshold = EPS_END + (EPS_START - EPS_END) *  np.exp(-1. * agentStep / EPS_DECAY)
            
            if  not train or (train and sample > eps_threshold): #var greedy
                QNetwork.eval()
                with torch.no_grad():
                    Q_value = QNetwork(self.statusFeature)
                    actionIndex = list((Q_value[:,1]-Q_value[:,0]).argsort()[0:self.Bu])
                    #actionIndex = list(Q_value.squeeze(1).argsort(descending=True)[0:self.Bu])
                QNetwork.train()
            else:
                actionIndex = list(torch.randint(0,self.contentNum,(self.Bu,)))

            self.action = torch.zeros(size=(env.contentNum,),dtype=int)
            self.action[self.W[-1]] = 1
            if self.W[-1] not in actionIndex:
                actionIndex.pop()
            for index in actionIndex:
                self.action[index] = 1
            env.updateEnv(self.u,self.action.numpy(),uit[2])
        else:
            self.action = self.lastAction # keep the cache and no request the new video

        self.reward = self.getReward(self.lastr[self.u],self.lastp,self.r[self.u],self.p,self.W[-1],self.lastAction,self.S,self.l,self.e,self.v)

        if train:
            lastAction = torch.cat(((1-self.lastAction).unsqueeze(1),self.lastAction.unsqueeze(1)),1)
            #lastAction = self.lastAction
            memory.push(self.lastStatusFeature, 
                        lastAction.to(device), 
                        self.statusFeature,
                        torch.tensor([self.reward.float()]).to(device),
                        torch.tensor([self.W[-1]]).to(device))
                        
        return self.action

In [6]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        #print(len(memory))
        return 0
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    #non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)), device=device, dtype=torch.bool)
    #non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)
    content_batch = torch.cat(batch.content)
    next_state_batch = torch.cat(batch.next_state)
    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_mask_bacth = action_batch.ge(0.5)
    state_action_values = torch.stack(torch.masked_select(policy_net(state_batch),state_action_mask_bacth).chunk(BATCH_SIZE,dim=0)).sum(dim=1)
    
    #state_action_values = torch.stack((policy_net(state_batch).squeeze(1) * action_batch).chunk(BATCH_SIZE,dim=0)).sum(dim=1)

    #print(state_action_values,state_action_values.dtype)
    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    #next_state_values = torch.zeros(BATCH_SIZE, device=device)
    #next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    '''
    def getNextStatusQ(s_batch,c_batch):
        
        Q_value= torch.stack(target_net(s_batch).detach().chunk(BATCH_SIZE,dim=0))
        c = c_batch.chunk(BATCH_SIZE,dim=0)
        action = torch.zeros(size=(BATCH_SIZE,contentNum,2),dtype=int).to(device)
        for b in range(BATCH_SIZE):
            Q_value_sortindex = list((Q_value[b,:,1]-Q_value[b,:,0]).argsort(descending=True)[0:Bu])
            i = c[b].squeeze()
            if i not in Q_value_sortindex:
                Q_value_sortindex.pop()
            action[b,i,1] = 1
            for index in Q_value_sortindex:
                action[b,index,1] = 1
        action[:,:,0]=1-action[:,:,1]
        action_mask = action.ge(0.5).to(device)
        next_state_values = torch.stack(torch.masked_select(Q_value,action_mask).chunk(BATCH_SIZE,dim=0)).sum(dim=1).float()

        return next_state_values
    '''
    #next_state_values =  getNextStatusQ(next_state_batch,content_batch)

    # Compute the expected Q values
    #expected_state_action_values = (next_state_values * GAMMA) + reward_batch
    expected_state_action_values =  reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values.unsqueeze(1), expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

    return loss

In [7]:
# Get screen size so that we can initialize layers correctly based on shape
# returned from AI gym. Typical dimensions at this point are close to 3x40x90
# which is the result of a clamped and down-scaled render buffer in get_screen()
#init_screen = get_screen()
#_, _, screen_height, screen_width = init_screen.shape
# Get number of actions from gym action space
#n_actions = env.action_space.n
policy_net = DQN(3, 2).to(device)
target_net = DQN(3, 2).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()
optimizer = optim.Adam(policy_net.parameters())
memory = ReplayMemory(10000)
torch.cuda.empty_cache()

bestReward =  float("-inf")
eps_threshold = 0.2

for i_episode in range(num_episodes):
    # Initialize the environment and state
    env = ENV(userNum,contentNum,latency,Bu)
    UEs = {}
    sumReward = np.zeros(3)
    loss = 0
    UEHit = np.zeros(userNum)
    edgeHit = 0
    for index,trace in trainUIT.iterrows():
        uit = trace.to_numpy()

        if uit[0] not in UEs:
            UEs[uit[0]] = UE(uit[0],env,rewardPara)
        ue = UEs[uit[0]]

        actionIndex = np.argwhere(ue.action)
        if uit[1] in actionIndex:
            UEHit[uit[0]] += 1
        elif uit[1] in env.pipe.keys():
            edgeHit += 1

        ue.selectAction(env,uit,policy_net,1,memory)

        sumReward[0] += float(ue.Rh)
        sumReward[1] += float(ue.Ro)
        sumReward[2] += float(ue.Rl)
        #agentStep += 1

        # train the DQN network
        if index % 25 == 0:
            # Perform one step of the optimization (on the policy network)
            loss += float(optimize_model())
        if index % 100 == 0 :
            # Update the target network, copying all weights and biases in DQN
            target_net.load_state_dict(policy_net.state_dict())
    psi = 0
    p = torch.from_numpy(env.p)
    for u in UEs:
        psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
    print("------------------------------train----------------------------------")
    print("--Time:",time.asctime( time.localtime(time.time())),"Episode:",i_episode,"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
    print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
    print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
    print("------------------------------train----------------------------------")
    print()
    # Update the target network, copying all weights and biases in DQN
    target_net.load_state_dict(policy_net.state_dict())
    
    #valid the hyperparameters
    env = ENV(userNum,contentNum,latency,Bu)
    UEs = {}
    sumReward = np.zeros(3)
    loss = 0
    UEHit = np.zeros(userNum)
    edgeHit = 0
    for index,trace in validUIT.iterrows():
        uit = trace.to_numpy()

        if uit[0] not in UEs:
            UEs[uit[0]] = UE(uit[0],env,rewardPara)
        ue = UEs[uit[0]]

        actionIndex = np.argwhere(ue.action)
        if uit[1] in actionIndex:
            UEHit[uit[0]] += 1
        elif uit[1] in env.pipe.keys():
            edgeHit += 1

        ue.selectAction(env,uit,policy_net,0,memory)

        sumReward[0] += float(ue.Rh)
        sumReward[1] += float(ue.Ro)
        sumReward[2] += float(ue.Rl)       
    psi = 0
    p = torch.from_numpy(env.p)
    for u in UEs:
        psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
    print("------------------------------validation----------------------------------")
    print("--Time:",time.asctime( time.localtime(time.time())),"Episode:",i_episode,"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
    print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
    print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
    print("------------------------------validation----------------------------------")
    print()
    modelSavePath = MODELPATH+'ep{}_'.format(i_episode)+time.strftime("%m%d-%H-%M-%S",time.localtime(time.time()))
    torch.save(policy_net.state_dict(),modelSavePath)
    if sumReward.sum() > bestReward:
        bestReward = sumReward.sum()
        bestEpisode = i_episode
        bestPath = modelSavePath
        print()
        print(bestPath)
        print()

------------------------------train----------------------------------
--Time: Mon Oct 11 00:52:37 2021 Episode: 0   Index: 65335   Loss: 0.10838 --
Reward: [-2.22255  0.       0.     ] total reward: -2.22255
UEHitrate: 0.01387  edgeHitrate 0.41385 sumHitrate 0.42771  privacy: 1.51748
------------------------------train----------------------------------

------------------------------validation----------------------------------
--Time: Mon Oct 11 00:55:04 2021 Episode: 0   Index: 81113   Loss: 0.0 --
Reward: [-3.20668  0.       0.     ] total reward: -3.20668
UEHitrate: 0.00693  edgeHitrate 0.30209 sumHitrate 0.30902  privacy: 0.78546
------------------------------validation----------------------------------


/home/ubuntu/data/PEC/model_dict/ml100k/dnn_o0l0_v5_ep0_1011-00-55-04

------------------------------train----------------------------------
--Time: Mon Oct 11 00:57:18 2021 Episode: 1   Index: 65335   Loss: 0.13592 --
Reward: [-3.23302  0.       0.     ] total reward: -3.23302
UE

In [8]:
#test
policy_net = DQN(3, 2).to(device)
target_net = DQN(3, 2).to(device)
policy_net.load_state_dict(torch.load(bestPath))
policy_net.eval()
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()
torch.cuda.empty_cache()


bestReward =  float("-inf")
env = ENV(userNum,contentNum,latency,Bu)
UEs = {}
sumReward = np.zeros(3)
loss = 0
UEHit = np.zeros(userNum)
edgeHit = 0

sumHitrate = np.zeros(UIT.shape[0] // 10000 +1)
UEHitrate = np.zeros(UIT.shape[0] // 10000 +1)
edgeHitrate = np.zeros(UIT.shape[0] // 10000 +1)
privacyReduction = np.zeros(UIT.shape[0] // 10000 +1)

for index,trace in UIT.iterrows():
    uit = trace.to_numpy()
    if uit[0] not in UEs:
        UEs[uit[0]] = UE(uit[0],env,rewardPara)
    ue = UEs[uit[0]]
    
    actionIndex = np.argwhere(ue.lastAction)
    if uit[1] in actionIndex:
        UEHit[uit[0]] += 1
    elif uit[1] in env.pipe.keys():
        edgeHit += 1
    ue.selectAction(env,uit,policy_net,0,memory)
    
    sumReward[0] += float(ue.Rh)
    sumReward[1] += float(ue.Ro)
    sumReward[2] += float(ue.Rl)
     
    if (index+1) % 10000 == 0 :
        psi = 0
        p = torch.from_numpy(env.p)
        for u in UEs:
            psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
        print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
        print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
        print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
        print()
        sumHitrate[int(index // 10000)]   = round((edgeHit+UEHit.sum())/(index+1),5)
        UEHitrate [int(index // 10000)]   = round(UEHit.sum()/(index+1),5)
        edgeHitrate [int(index // 10000)] = round(edgeHit/(index+1),5)
        privacyReduction [int(index // 10000)] = round(float(psi)/len(UEs),5)
        
psi = 0
p = torch.from_numpy(env.p)
for u in UEs:
    psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()

print()
print("----------------------------------------------------------------")
print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
print("----------------------------------------------------------------")
print()
sumHitrate [int(round(index / 10000,0))]  = round((edgeHit+UEHit.sum())/(index+1),5)
UEHitrate  [int(round(index / 10000,0))]  = round(UEHit.sum()/(index+1),5)
edgeHitrate[int(round(index / 10000,0))]  = round(edgeHit/(index+1),5)
privacyReduction [int(round(index / 10000,0))] = round(float(psi)/len(UEs),5)

--Time: Mon Oct 11 02:27:03 2021   Index: 9999   Loss: 0.0 --
Reward: [-5.44128  0.       0.     ] total reward: -5.44128
UEHitrate: 0.0077  edgeHitrate 0.2742 sumHitrate 0.2819  privacy: 6.62953

--Time: Mon Oct 11 02:27:21 2021   Index: 19999   Loss: 0.0 --
Reward: [-4.95775  0.       0.     ] total reward: -4.95775
UEHitrate: 0.0082  edgeHitrate 0.2885 sumHitrate 0.2967  privacy: 4.38349

--Time: Mon Oct 11 02:27:39 2021   Index: 29999   Loss: 0.0 --
Reward: [-3.73797  0.       0.     ] total reward: -3.73797
UEHitrate: 0.00737  edgeHitrate 0.30837 sumHitrate 0.31573  privacy: 3.06879

--Time: Mon Oct 11 02:27:58 2021   Index: 39999   Loss: 0.0 --
Reward: [-3.55144  0.       0.     ] total reward: -3.55144
UEHitrate: 0.00692  edgeHitrate 0.30345 sumHitrate 0.31038  privacy: 2.38948

--Time: Mon Oct 11 02:28:17 2021   Index: 49999   Loss: 0.0 --
Reward: [-3.27824  0.       0.     ] total reward: -3.27824
UEHitrate: 0.00628  edgeHitrate 0.3006 sumHitrate 0.30688  privacy: 1.91545

--T

In [9]:
sumHitrate, UEHitrate, edgeHitrate

(array([0.2819 , 0.2967 , 0.31573, 0.31038, 0.30688, 0.30815, 0.3238 ,
        0.35865, 0.39639, 0.41824, 0.41824]),
 array([0.0077 , 0.0082 , 0.00737, 0.00692, 0.00628, 0.00598, 0.00631,
        0.00651, 0.00681, 0.00699, 0.00699]),
 array([0.2742 , 0.2885 , 0.30837, 0.30345, 0.3006 , 0.30217, 0.31749,
        0.35214, 0.38958, 0.41125, 0.41125]))

In [10]:
privacyReduction

array([6.62953, 4.38349, 3.06879, 2.38948, 1.91545, 1.4927 , 1.3129 ,
       1.24267, 1.15457, 1.09798, 1.09798])

In [11]:
bestPath

'/home/ubuntu/data/PEC/model_dict/ml100k/dnn_o0l0_v5_ep7_1011-01-28-04'

In [12]:
# Get screen size so that we can initialize layers correctly based on shape
# returned from AI gym. Typical dimensions at this point are close to 3x40x90
# which is the result of a clamped and down-scaled render buffer in get_screen()
#init_screen = get_screen()
#_, _, screen_height, screen_width = init_screen.shape
# Get number of actions from gym action space
#n_actions = env.action_space.n
policy_net = DQN(3, 2).to(device)
target_net = DQN(3, 2).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()
optimizer = optim.Adam(policy_net.parameters())
memory = ReplayMemory(10000)
torch.cuda.empty_cache()

bestReward =  float("-inf")
eps_threshold = 0.5

for i_episode in range(num_episodes):
    # Initialize the environment and state
    env = ENV(userNum,contentNum,latency,Bu)
    UEs = {}
    sumReward = np.zeros(3)
    loss = 0
    UEHit = np.zeros(userNum)
    edgeHit = 0
    for index,trace in trainUIT.iterrows():
        uit = trace.to_numpy()

        if uit[0] not in UEs:
            UEs[uit[0]] = UE(uit[0],env,rewardPara)
        ue = UEs[uit[0]]

        actionIndex = np.argwhere(ue.action)
        if uit[1] in actionIndex:
            UEHit[uit[0]] += 1
        elif uit[1] in env.pipe.keys():
            edgeHit += 1

        ue.selectAction(env,uit,policy_net,1,memory)

        sumReward[0] += float(ue.Rh)
        sumReward[1] += float(ue.Ro)
        sumReward[2] += float(ue.Rl)
        #agentStep += 1

        # train the DQN network
        if index % 25 == 0:
            # Perform one step of the optimization (on the policy network)
            loss += float(optimize_model())
        if index % 100 == 0 :
            # Update the target network, copying all weights and biases in DQN
            target_net.load_state_dict(policy_net.state_dict())
    psi = 0
    p = torch.from_numpy(env.p)
    for u in UEs:
        psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
    print("------------------------------train----------------------------------")
    print("--Time:",time.asctime( time.localtime(time.time())),"Episode:",i_episode,"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
    print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
    print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
    print("------------------------------train----------------------------------")
    print()
    # Update the target network, copying all weights and biases in DQN
    target_net.load_state_dict(policy_net.state_dict())
    
    #valid the hyperparameters
    env = ENV(userNum,contentNum,latency,Bu)
    UEs = {}
    sumReward = np.zeros(3)
    loss = 0
    UEHit = np.zeros(userNum)
    edgeHit = 0
    for index,trace in validUIT.iterrows():
        uit = trace.to_numpy()

        if uit[0] not in UEs:
            UEs[uit[0]] = UE(uit[0],env,rewardPara)
        ue = UEs[uit[0]]

        actionIndex = np.argwhere(ue.action)
        if uit[1] in actionIndex:
            UEHit[uit[0]] += 1
        elif uit[1] in env.pipe.keys():
            edgeHit += 1

        ue.selectAction(env,uit,policy_net,0,memory)

        sumReward[0] += float(ue.Rh)
        sumReward[1] += float(ue.Ro)
        sumReward[2] += float(ue.Rl)       
    psi = 0
    p = torch.from_numpy(env.p)
    for u in UEs:
        psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
    print("------------------------------validation----------------------------------")
    print("--Time:",time.asctime( time.localtime(time.time())),"Episode:",i_episode,"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
    print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
    print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
    print("------------------------------validation----------------------------------")
    print()
    modelSavePath = MODELPATH+'ep{}_'.format(i_episode)+time.strftime("%m%d-%H-%M-%S",time.localtime(time.time()))
    torch.save(policy_net.state_dict(),modelSavePath)
    if sumReward.sum() > bestReward:
        bestReward = sumReward.sum()
        bestEpisode = i_episode
        bestPath = modelSavePath
        print()
        print(bestPath)
        print()

------------------------------train----------------------------------
--Time: Mon Oct 11 02:31:55 2021 Episode: 0   Index: 65335   Loss: 0.20758 --
Reward: [-3.02872  0.       0.     ] total reward: -3.02872
UEHitrate: 0.01081  edgeHitrate 0.34156 sumHitrate 0.35236  privacy: 1.32102
------------------------------train----------------------------------

------------------------------validation----------------------------------
--Time: Mon Oct 11 02:34:58 2021 Episode: 0   Index: 81113   Loss: 0.0 --
Reward: [-3.2824  0.      0.    ] total reward: -3.2824
UEHitrate: 0.00795  edgeHitrate 0.30494 sumHitrate 0.31289  privacy: 0.74002
------------------------------validation----------------------------------


/home/ubuntu/data/PEC/model_dict/ml100k/dnn_o0l0_v5_ep0_1011-02-34-58

------------------------------train----------------------------------
--Time: Mon Oct 11 02:36:58 2021 Episode: 1   Index: 65335   Loss: 0.16849 --
Reward: [-3.08873  0.       0.     ] total reward: -3.08873
UEHitr

In [13]:
#test
policy_net = DQN(3, 2).to(device)
target_net = DQN(3, 2).to(device)
policy_net.load_state_dict(torch.load(bestPath))
policy_net.eval()
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()
torch.cuda.empty_cache()


bestReward =  float("-inf")
env = ENV(userNum,contentNum,latency,Bu)
UEs = {}
sumReward = np.zeros(3)
loss = 0
UEHit = np.zeros(userNum)
edgeHit = 0

sumHitrate = np.zeros(UIT.shape[0] // 10000 +1)
UEHitrate = np.zeros(UIT.shape[0] // 10000 +1)
edgeHitrate = np.zeros(UIT.shape[0] // 10000 +1)
privacyReduction = np.zeros(UIT.shape[0] // 10000 +1)

for index,trace in UIT.iterrows():
    uit = trace.to_numpy()
    if uit[0] not in UEs:
        UEs[uit[0]] = UE(uit[0],env,rewardPara)
    ue = UEs[uit[0]]
    
    actionIndex = np.argwhere(ue.lastAction)
    if uit[1] in actionIndex:
        UEHit[uit[0]] += 1
    elif uit[1] in env.pipe.keys():
        edgeHit += 1
    ue.selectAction(env,uit,policy_net,0,memory)
    
    sumReward[0] += float(ue.Rh)
    sumReward[1] += float(ue.Ro)
    sumReward[2] += float(ue.Rl)
     
    if (index+1) % 10000 == 0 :
        psi = 0
        p = torch.from_numpy(env.p)
        for u in UEs:
            psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
        print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
        print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
        print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
        print()
        sumHitrate[int(index // 10000)]   = round((edgeHit+UEHit.sum())/(index+1),5)
        UEHitrate [int(index // 10000)]   = round(UEHit.sum()/(index+1),5)
        edgeHitrate [int(index // 10000)] = round(edgeHit/(index+1),5)
        privacyReduction [int(index // 10000)] = round(float(psi)/len(UEs),5)
        
psi = 0
p = torch.from_numpy(env.p)
for u in UEs:
    psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()

print()
print("----------------------------------------------------------------")
print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
print("----------------------------------------------------------------")
print()
sumHitrate [int(round(index / 10000,0))]  = round((edgeHit+UEHit.sum())/(index+1),5)
UEHitrate  [int(round(index / 10000,0))]  = round(UEHit.sum()/(index+1),5)
edgeHitrate[int(round(index / 10000,0))]  = round(edgeHit/(index+1),5)
privacyReduction [int(round(index / 10000,0))] = round(float(psi)/len(UEs),5)

--Time: Mon Oct 11 04:06:41 2021   Index: 9999   Loss: 0.0 --
Reward: [-5.44603  0.       0.     ] total reward: -5.44603
UEHitrate: 0.0081  edgeHitrate 0.295 sumHitrate 0.3031  privacy: 6.65198

--Time: Mon Oct 11 04:07:01 2021   Index: 19999   Loss: 0.0 --
Reward: [-4.95855  0.       0.     ] total reward: -4.95855
UEHitrate: 0.00785  edgeHitrate 0.29845 sumHitrate 0.3063  privacy: 4.3751

--Time: Mon Oct 11 04:07:21 2021   Index: 29999   Loss: 0.0 --
Reward: [-3.73224  0.       0.     ] total reward: -3.73224
UEHitrate: 0.00777  edgeHitrate 0.29713 sumHitrate 0.3049  privacy: 2.84551

--Time: Mon Oct 11 04:07:41 2021   Index: 39999   Loss: 0.0 --
Reward: [-3.52528  0.       0.     ] total reward: -3.52528
UEHitrate: 0.0081  edgeHitrate 0.29885 sumHitrate 0.30695  privacy: 2.11572

--Time: Mon Oct 11 04:08:01 2021   Index: 49999   Loss: 0.0 --
Reward: [-3.2438  0.      0.    ] total reward: -3.2438
UEHitrate: 0.00786  edgeHitrate 0.30042 sumHitrate 0.30828  privacy: 1.65136

--Time: 

In [14]:
sumHitrate, UEHitrate, edgeHitrate

(array([0.3031 , 0.3063 , 0.3049 , 0.30695, 0.30828, 0.31292, 0.31404,
        0.31254, 0.3168 , 0.31894, 0.31894]),
 array([0.0081 , 0.00785, 0.00777, 0.0081 , 0.00786, 0.00787, 0.00773,
        0.00748, 0.00762, 0.00755, 0.00755]),
 array([0.295  , 0.29845, 0.29713, 0.29885, 0.30042, 0.30505, 0.30631,
        0.30506, 0.30918, 0.31139, 0.31139]))

In [15]:
privacyReduction

array([6.65198, 4.3751 , 2.84551, 2.11572, 1.65136, 1.22643, 0.95316,
       0.76447, 0.539  , 0.29245, 0.29245])

In [16]:
bestPath

'/home/ubuntu/data/PEC/model_dict/ml100k/dnn_o0l0_v5_ep0_1011-02-34-58'

In [17]:
# Get screen size so that we can initialize layers correctly based on shape
# returned from AI gym. Typical dimensions at this point are close to 3x40x90
# which is the result of a clamped and down-scaled render buffer in get_screen()
#init_screen = get_screen()
#_, _, screen_height, screen_width = init_screen.shape
# Get number of actions from gym action space
#n_actions = env.action_space.n
policy_net = DQN(3, 2).to(device)
target_net = DQN(3, 2).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()
optimizer = optim.Adam(policy_net.parameters())
memory = ReplayMemory(10000)
torch.cuda.empty_cache()

bestReward =  float("-inf")
eps_threshold = 0.8

for i_episode in range(num_episodes):
    # Initialize the environment and state
    env = ENV(userNum,contentNum,latency,Bu)
    UEs = {}
    sumReward = np.zeros(3)
    loss = 0
    UEHit = np.zeros(userNum)
    edgeHit = 0
    for index,trace in trainUIT.iterrows():
        uit = trace.to_numpy()

        if uit[0] not in UEs:
            UEs[uit[0]] = UE(uit[0],env,rewardPara)
        ue = UEs[uit[0]]

        actionIndex = np.argwhere(ue.action)
        if uit[1] in actionIndex:
            UEHit[uit[0]] += 1
        elif uit[1] in env.pipe.keys():
            edgeHit += 1

        ue.selectAction(env,uit,policy_net,1,memory)

        sumReward[0] += float(ue.Rh)
        sumReward[1] += float(ue.Ro)
        sumReward[2] += float(ue.Rl)
        #agentStep += 1

        # train the DQN network
        if index % 25 == 0:
            # Perform one step of the optimization (on the policy network)
            loss += float(optimize_model())
        if index % 100 == 0 :
            # Update the target network, copying all weights and biases in DQN
            target_net.load_state_dict(policy_net.state_dict())
    psi = 0
    p = torch.from_numpy(env.p)
    for u in UEs:
        psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
    print("------------------------------train----------------------------------")
    print("--Time:",time.asctime( time.localtime(time.time())),"Episode:",i_episode,"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
    print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
    print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
    print("------------------------------train----------------------------------")
    print()
    # Update the target network, copying all weights and biases in DQN
    target_net.load_state_dict(policy_net.state_dict())
    
    #valid the hyperparameters
    env = ENV(userNum,contentNum,latency,Bu)
    UEs = {}
    sumReward = np.zeros(3)
    loss = 0
    UEHit = np.zeros(userNum)
    edgeHit = 0
    for index,trace in validUIT.iterrows():
        uit = trace.to_numpy()

        if uit[0] not in UEs:
            UEs[uit[0]] = UE(uit[0],env,rewardPara)
        ue = UEs[uit[0]]

        actionIndex = np.argwhere(ue.action)
        if uit[1] in actionIndex:
            UEHit[uit[0]] += 1
        elif uit[1] in env.pipe.keys():
            edgeHit += 1

        ue.selectAction(env,uit,policy_net,0,memory)

        sumReward[0] += float(ue.Rh)
        sumReward[1] += float(ue.Ro)
        sumReward[2] += float(ue.Rl)       
    psi = 0
    p = torch.from_numpy(env.p)
    for u in UEs:
        psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
    print("------------------------------validation----------------------------------")
    print("--Time:",time.asctime( time.localtime(time.time())),"Episode:",i_episode,"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
    print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
    print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
    print("------------------------------validation----------------------------------")
    print()
    modelSavePath = MODELPATH+'ep{}_'.format(i_episode)+time.strftime("%m%d-%H-%M-%S",time.localtime(time.time()))
    torch.save(policy_net.state_dict(),modelSavePath)
    if sumReward.sum() > bestReward:
        bestReward = sumReward.sum()
        bestEpisode = i_episode
        bestPath = modelSavePath
        print()
        print(bestPath)
        print()

------------------------------train----------------------------------
--Time: Mon Oct 11 04:11:20 2021 Episode: 0   Index: 65335   Loss: 0.30786 --
Reward: [-2.25285  0.       0.     ] total reward: -2.25285
UEHitrate: 0.01163  edgeHitrate 0.30631 sumHitrate 0.31794  privacy: 1.80176
------------------------------train----------------------------------

------------------------------validation----------------------------------
--Time: Mon Oct 11 04:14:03 2021 Episode: 0   Index: 81113   Loss: 0.0 --
Reward: [-0.35285  0.       0.     ] total reward: -0.35285
UEHitrate: 0.00153  edgeHitrate 0.49615 sumHitrate 0.49768  privacy: 1.22871
------------------------------validation----------------------------------


/home/ubuntu/data/PEC/model_dict/ml100k/dnn_o0l0_v5_ep0_1011-04-14-03

------------------------------train----------------------------------
--Time: Mon Oct 11 04:15:47 2021 Episode: 1   Index: 65335   Loss: 0.28103 --
Reward: [-2.26133  0.       0.     ] total reward: -2.26133
UE

In [ ]:
#test
policy_net = DQN(3, 2).to(device)
target_net = DQN(3, 2).to(device)
policy_net.load_state_dict(torch.load(bestPath))
policy_net.eval()
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()
torch.cuda.empty_cache()


bestReward =  float("-inf")
env = ENV(userNum,contentNum,latency,Bu)
UEs = {}
sumReward = np.zeros(3)
loss = 0
UEHit = np.zeros(userNum)
edgeHit = 0

sumHitrate = np.zeros(UIT.shape[0] // 10000 +1)
UEHitrate = np.zeros(UIT.shape[0] // 10000 +1)
edgeHitrate = np.zeros(UIT.shape[0] // 10000 +1)
privacyReduction = np.zeros(UIT.shape[0] // 10000 +1)

for index,trace in UIT.iterrows():
    uit = trace.to_numpy()
    if uit[0] not in UEs:
        UEs[uit[0]] = UE(uit[0],env,rewardPara)
    ue = UEs[uit[0]]
    
    actionIndex = np.argwhere(ue.lastAction)
    if uit[1] in actionIndex:
        UEHit[uit[0]] += 1
    elif uit[1] in env.pipe.keys():
        edgeHit += 1
    ue.selectAction(env,uit,policy_net,0,memory)
    
    sumReward[0] += float(ue.Rh)
    sumReward[1] += float(ue.Ro)
    sumReward[2] += float(ue.Rl)
     
    if (index+1) % 10000 == 0 :
        psi = 0
        p = torch.from_numpy(env.p)
        for u in UEs:
            psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
        print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
        print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
        print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
        print()
        sumHitrate[int(index // 10000)]   = round((edgeHit+UEHit.sum())/(index+1),5)
        UEHitrate [int(index // 10000)]   = round(UEHit.sum()/(index+1),5)
        edgeHitrate [int(index // 10000)] = round(edgeHit/(index+1),5)
        privacyReduction [int(index // 10000)] = round(float(psi)/len(UEs),5)
        
psi = 0
p = torch.from_numpy(env.p)
for u in UEs:
    psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()

print()
print("----------------------------------------------------------------")
print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
print("----------------------------------------------------------------")
print()
sumHitrate [int(round(index / 10000,0))]  = round((edgeHit+UEHit.sum())/(index+1),5)
UEHitrate  [int(round(index / 10000,0))]  = round(UEHit.sum()/(index+1),5)
edgeHitrate[int(round(index / 10000,0))]  = round(edgeHit/(index+1),5)
privacyReduction [int(round(index / 10000,0))] = round(float(psi)/len(UEs),5)

--Time: Mon Oct 11 05:40:22 2021   Index: 9999   Loss: 0.0 --
Reward: [-0.57807  0.       0.     ] total reward: -0.57807
UEHitrate: 0.0022  edgeHitrate 0.5069 sumHitrate 0.5091  privacy: 1.58945

--Time: Mon Oct 11 05:40:42 2021   Index: 19999   Loss: 0.0 --
Reward: [-0.53477  0.       0.     ] total reward: -0.53477
UEHitrate: 0.0018  edgeHitrate 0.52675 sumHitrate 0.52855  privacy: 1.43453

--Time: Mon Oct 11 05:41:02 2021   Index: 29999   Loss: 0.0 --
Reward: [-0.42061  0.       0.     ] total reward: -0.42061
UEHitrate: 0.00173  edgeHitrate 0.53437 sumHitrate 0.5361  privacy: 1.37822

--Time: Mon Oct 11 05:41:23 2021   Index: 39999   Loss: 0.0 --
Reward: [-0.39449  0.       0.     ] total reward: -0.39449
UEHitrate: 0.00172  edgeHitrate 0.53373 sumHitrate 0.53545  privacy: 1.32796

--Time: Mon Oct 11 05:41:43 2021   Index: 49999   Loss: 0.0 --
Reward: [-0.368  0.     0.   ] total reward: -0.368
UEHitrate: 0.00158  edgeHitrate 0.52516 sumHitrate 0.52674  privacy: 1.29221

--Time: M

In [ ]:
sumHitrate, UEHitrate, edgeHitrate

(array([0.5091 , 0.52855, 0.5361 , 0.53545, 0.52674, 0.52137, 0.50864,
        0.49839, 0.5033 , 0.49922, 0.49922]),
 array([0.0022 , 0.0018 , 0.00173, 0.00172, 0.00158, 0.00148, 0.0015 ,
        0.00148, 0.00143, 0.00144, 0.00144]),
 array([0.5069 , 0.52675, 0.53437, 0.53373, 0.52516, 0.51988, 0.50714,
        0.49691, 0.50187, 0.49778, 0.49778]))

In [ ]:
privacyReduction

array([1.58945, 1.43453, 1.37822, 1.32796, 1.29221, 1.27062, 1.2488 ,
       1.23101, 1.2165 , 1.20142, 1.20142])

In [ ]:
bestPath

'/home/ubuntu/data/PEC/model_dict/ml100k/dnn_o0l0_v5_ep0_1011-04-14-03'

In [22]:
# Get screen size so that we can initialize layers correctly based on shape
# returned from AI gym. Typical dimensions at this point are close to 3x40x90
# which is the result of a clamped and down-scaled render buffer in get_screen()
#init_screen = get_screen()
#_, _, screen_height, screen_width = init_screen.shape
# Get number of actions from gym action space
#n_actions = env.action_space.n
policy_net = DQN(3, 2).to(device)
target_net = DQN(3, 2).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()
optimizer = optim.Adam(policy_net.parameters())
memory = ReplayMemory(10000)
torch.cuda.empty_cache()

bestReward =  float("-inf")
eps_threshold = 0

for i_episode in range(num_episodes):
    # Initialize the environment and state
    env = ENV(userNum,contentNum,latency,Bu)
    UEs = {}
    sumReward = np.zeros(3)
    loss = 0
    UEHit = np.zeros(userNum)
    edgeHit = 0
    for index,trace in trainUIT.iterrows():
        uit = trace.to_numpy()

        if uit[0] not in UEs:
            UEs[uit[0]] = UE(uit[0],env,rewardPara)
        ue = UEs[uit[0]]

        actionIndex = np.argwhere(ue.action)
        if uit[1] in actionIndex:
            UEHit[uit[0]] += 1
        elif uit[1] in env.pipe.keys():
            edgeHit += 1

        ue.selectAction(env,uit,policy_net,1,memory)

        sumReward[0] += float(ue.Rh)
        sumReward[1] += float(ue.Ro)
        sumReward[2] += float(ue.Rl)
        #agentStep += 1

        # train the DQN network
        if index % 25 == 0:
            # Perform one step of the optimization (on the policy network)
            loss += float(optimize_model())
        if index % 100 == 0 :
            # Update the target network, copying all weights and biases in DQN
            target_net.load_state_dict(policy_net.state_dict())
    psi = 0
    p = torch.from_numpy(env.p)
    for u in UEs:
        psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
    print("------------------------------train----------------------------------")
    print("--Time:",time.asctime( time.localtime(time.time())),"Episode:",i_episode,"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
    print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
    print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
    print("------------------------------train----------------------------------")
    print()
    # Update the target network, copying all weights and biases in DQN
    target_net.load_state_dict(policy_net.state_dict())
    
    #valid the hyperparameters
    env = ENV(userNum,contentNum,latency,Bu)
    UEs = {}
    sumReward = np.zeros(3)
    loss = 0
    UEHit = np.zeros(userNum)
    edgeHit = 0
    for index,trace in validUIT.iterrows():
        uit = trace.to_numpy()

        if uit[0] not in UEs:
            UEs[uit[0]] = UE(uit[0],env,rewardPara)
        ue = UEs[uit[0]]

        actionIndex = np.argwhere(ue.action)
        if uit[1] in actionIndex:
            UEHit[uit[0]] += 1
        elif uit[1] in env.pipe.keys():
            edgeHit += 1

        ue.selectAction(env,uit,policy_net,0,memory)

        sumReward[0] += float(ue.Rh)
        sumReward[1] += float(ue.Ro)
        sumReward[2] += float(ue.Rl)       
    psi = 0
    p = torch.from_numpy(env.p)
    for u in UEs:
        psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
    print("------------------------------validation----------------------------------")
    print("--Time:",time.asctime( time.localtime(time.time())),"Episode:",i_episode,"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
    print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
    print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
    print("------------------------------validation----------------------------------")
    print()
    modelSavePath = MODELPATH+'ep{}_'.format(i_episode)+time.strftime("%m%d-%H-%M-%S",time.localtime(time.time()))
    torch.save(policy_net.state_dict(),modelSavePath)
    if sumReward.sum() > bestReward:
        bestReward = sumReward.sum()
        bestEpisode = i_episode
        bestPath = modelSavePath
        print()
        print(bestPath)
        print()

------------------------------train----------------------------------
--Time: Mon Oct 11 10:23:22 2021 Episode: 0   Index: 65335   Loss: 0.15375 --
Reward: [-0.36646  0.       0.     ] total reward: -0.36646
UEHitrate: 0.00099  edgeHitrate 0.49761 sumHitrate 0.49861  privacy: 1.25826
------------------------------train----------------------------------

------------------------------validation----------------------------------
--Time: Mon Oct 11 10:26:02 2021 Episode: 0   Index: 81113   Loss: 0.0 --
Reward: [-0.35069  0.       0.     ] total reward: -0.35069
UEHitrate: 0.00104  edgeHitrate 0.48101 sumHitrate 0.48205  privacy: 1.22761
------------------------------validation----------------------------------


/home/ubuntu/data/PEC/model_dict/ml100k/dnn_o0l0_v5_ep0_1011-10-26-02

------------------------------train----------------------------------
--Time: Mon Oct 11 10:28:39 2021 Episode: 1   Index: 65335   Loss: 0.11375 --
Reward: [-0.36635  0.       0.     ] total reward: -0.36635
UE

In [23]:
#test
policy_net = DQN(3, 2).to(device)
target_net = DQN(3, 2).to(device)
policy_net.load_state_dict(torch.load(bestPath))
policy_net.eval()
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()
torch.cuda.empty_cache()


bestReward =  float("-inf")
env = ENV(userNum,contentNum,latency,Bu)
UEs = {}
sumReward = np.zeros(3)
loss = 0
UEHit = np.zeros(userNum)
edgeHit = 0

sumHitrate = np.zeros(UIT.shape[0] // 10000 +1)
UEHitrate = np.zeros(UIT.shape[0] // 10000 +1)
edgeHitrate = np.zeros(UIT.shape[0] // 10000 +1)
privacyReduction = np.zeros(UIT.shape[0] // 10000 +1)

for index,trace in UIT.iterrows():
    uit = trace.to_numpy()
    if uit[0] not in UEs:
        UEs[uit[0]] = UE(uit[0],env,rewardPara)
    ue = UEs[uit[0]]
    
    actionIndex = np.argwhere(ue.lastAction)
    if uit[1] in actionIndex:
        UEHit[uit[0]] += 1
    elif uit[1] in env.pipe.keys():
        edgeHit += 1
    ue.selectAction(env,uit,policy_net,0,memory)
    
    sumReward[0] += float(ue.Rh)
    sumReward[1] += float(ue.Ro)
    sumReward[2] += float(ue.Rl)
     
    if (index+1) % 10000 == 0 :
        psi = 0
        p = torch.from_numpy(env.p)
        for u in UEs:
            psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
        print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
        print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
        print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
        print()
        sumHitrate[int(index // 10000)]   = round((edgeHit+UEHit.sum())/(index+1),5)
        UEHitrate [int(index // 10000)]   = round(UEHit.sum()/(index+1),5)
        edgeHitrate [int(index // 10000)] = round(edgeHit/(index+1),5)
        privacyReduction [int(index // 10000)] = round(float(psi)/len(UEs),5)
        
psi = 0
p = torch.from_numpy(env.p)
for u in UEs:
    psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()

print()
print("----------------------------------------------------------------")
print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
print("----------------------------------------------------------------")
print()
sumHitrate [int(round(index / 10000,0))]  = round((edgeHit+UEHit.sum())/(index+1),5)
UEHitrate  [int(round(index / 10000,0))]  = round(UEHit.sum()/(index+1),5)
edgeHitrate[int(round(index / 10000,0))]  = round(edgeHit/(index+1),5)
privacyReduction [int(round(index / 10000,0))] = round(float(psi)/len(UEs),5)

--Time: Mon Oct 11 14:26:44 2021   Index: 9999   Loss: 0.0 --
Reward: [-0.57783  0.       0.     ] total reward: -0.57783
UEHitrate: 0.0019  edgeHitrate 0.5071 sumHitrate 0.509  privacy: 1.59112

--Time: Mon Oct 11 14:27:05 2021   Index: 19999   Loss: 0.0 --
Reward: [-0.53051  0.       0.     ] total reward: -0.53051
UEHitrate: 0.0068  edgeHitrate 0.51075 sumHitrate 0.51755  privacy: 1.41753

--Time: Mon Oct 11 14:27:25 2021   Index: 29999   Loss: 0.0 --
Reward: [-0.4093  0.      0.    ] total reward: -0.4093
UEHitrate: 0.02463  edgeHitrate 0.5312 sumHitrate 0.55583  privacy: 1.26861

--Time: Mon Oct 11 14:27:45 2021   Index: 39999   Loss: 0.0 --
Reward: [-0.38171  0.       0.     ] total reward: -0.38171
UEHitrate: 0.03258  edgeHitrate 0.54053 sumHitrate 0.5731  privacy: 1.19273

--Time: Mon Oct 11 14:28:05 2021   Index: 49999   Loss: 0.0 --
Reward: [-0.35605  0.       0.     ] total reward: -0.35605
UEHitrate: 0.03662  edgeHitrate 0.53162 sumHitrate 0.56824  privacy: 1.14609

--Time:

In [24]:
sumHitrate, UEHitrate, edgeHitrate

(array([0.509  , 0.51755, 0.55583, 0.5731 , 0.56824, 0.56885, 0.55881,
        0.54858, 0.556  , 0.55219, 0.55219]),
 array([0.0019 , 0.0068 , 0.02463, 0.03258, 0.03662, 0.04087, 0.04266,
        0.04355, 0.04559, 0.04546, 0.04546]),
 array([0.5071 , 0.51075, 0.5312 , 0.54053, 0.53162, 0.52798, 0.51616,
        0.50502, 0.51041, 0.50673, 0.50673]))

In [25]:
privacyReduction

array([1.59112, 1.41753, 1.26861, 1.19273, 1.14609, 1.10224, 1.0713 ,
       1.04645, 1.013  , 0.98708, 0.98708])

In [26]:
bestPath

'/home/ubuntu/data/PEC/model_dict/ml100k/dnn_o0l0_v5_ep6_1011-10-57-41'